# TINA - Assistente virtual

<p align="left">
  <img src="https://img.shields.io/static/v1?label=&message=Python&color=blue&style=for-the-badge&logo=python"/>
  <img src="https://img.shields.io/static/v1?label=&message=gTTS&color=blue&style=for-the-badge&logo=gtts"/>
  <img src="https://img.shields.io/static/v1?label=&message=speech_recognition&color=blue&style=for-the-badge&logo=speech_recognition"/>
  <img src="http://img.shields.io/static/v1?label=STATUS&message=CONCLUIDO&color=GREEN&style=for-the-badge"/>
</p>


Este Projeto foi realizado por Maurício André de Almeida como trabalho no curso de Processamento de Linguagem Natural na DIO.ME


# Projeto Criando um sistema de assistência virtual do zero. 

Neste projeto deve ser desenvolvido um sistema de assistência virtual, utilizando PLN (Processamento de Linguagem Natural), com base nas bibliotecas apresentadas durante o curso. O sistema deve obedecer aos seguintes requisitos: 
	
Um módulo para transformação de texto em áudio (text to speech); 
 	
	
Um módulo para transformação de fala (linguagem natural humana) em texto (speech to text); 
 	
	
O módulo 2, deve acionar por comando de voz algumas funções automatizadas, como por exemplo: abrir uma pesquisa no Wikipedia, abrir o Youtube, apresentar a localização da farmácia mais próxima. 
 	
Para realizar este projeto, podem ser utilizada todas as bibliotecas apresentadas no curso, principalmente a biblioteca Speech recognition em Python.  



### <b>Implementação:</b>

Neste projeto utilizei as bibliotecas speech_recognition, pyaudio, pygame, gtts, webbrowser e subprocess

Realizei os seguintes passos para cumprir o desafio:

<ul>
<li>Carregar as bibliotecas necessárias</li>
<li>Criar a função para falar um texto</li>
<li>Criar as strings do nome da assistente, mensagens padrão e traduções para tempo e clima</li>
<li>Função para remover verbos dos comandos</li>
<li>Função para pesquisar a previsão do tempo</li>
<li>Função para processar os comandos e executar as ações</li>
<ul>
  <li>Pesquisar na internet</li>
  <li>Pesquisar na Wikipedia</li>
  <li>Pesquisar no Youtube</li>
  <li>Falar a data</li>
  <li>Falar a hora</li>
  <li>Falar a previsão do tempo</li>
</ul>
<li>Função para ouvir o microfone</li>
<li>Programa principal que escuta a palavra de ativação e manda executar os comandos até ouvir a palavra "Sair"</li>
</ul>

## Importação das bibliotecas

In [9]:
import speech_recognition as sr
import pyaudio as audio
import io
import datetime
import requests
import pygame
from gtts import gTTS
import webbrowser
import subprocess
import re

## Função para falar um texto

In [10]:

def falar(texto):
    # Print apenas para efeito de debug
    print(f'falando: {texto}')
    # Cria um objeto gTTS com o texto fornecido
    tts = gTTS(text=texto, lang='pt-br', slow = False)
    # Salva o áudio em um buffer de memória (BytesIO)
    audio_buffer = io.BytesIO()
    tts.write_to_fp(audio_buffer)
    audio_buffer.seek(0)  # Volta ao início do buffer

    # Inicializa o pygame para reprodução de áudio
    pygame.mixer.init()
    pygame.mixer.music.load(audio_buffer)
    pygame.mixer.music.play()

    # Espera a reprodução terminar
    while pygame.mixer.music.get_busy():
        pygame.time.Clock().tick(10)


## Definição de strings padrão e traduções

In [11]:
# Nome da assistente (gatilho)
nome_assistente = 'Tina'

frase_ola = 'Olá sou '+nome_assistente+' sua assistente virtual'
frase_ajuda = 'Para que eu possa responder seus comandos, inicie uma frase com: '+nome_assistente+'!' 
frase_ajuda2 = 'Por exemplo, diga: '+nome_assistente+', que horas são?'
frase_aguarde = 'aguarde um instante...'
frase_ajudar = 'em que posso ajudar?'
frase_erro_comando = 'Comando não reconhecido. Tente novamente.'
frase_nao_entendi = 'Não entendi! pode repetir?'
frase_erro = 'Desculpe! Ocorreu um erro em meu processamento, tente novamente mais tarde!'
frase_encerrando = 'Encerrando, tenha um ótimo dia!'


# Lista de verbos comuns para remover
verbos = ['abra', 'abre', 'abrir', 'pesquise', 'pesquisar', 'encontre', 'encontrar', 'procure', 'procurar', 'mostre', 'mostrar',
            'sobre', 'inicia', 'iniciar' ]

# Dicionários para traduzir dias e meses para português
DIAS_PT = {
    'Monday': 'segunda-feira',
    'Tuesday': 'terça-feira',
    'Wednesday': 'quarta-feira',
    'Thursday': 'quinta-feira',
    'Friday': 'sexta-feira',
    'Saturday': 'sábado',
    'Sunday': 'domingo'
}

MESES_PT = {
    'January': 'janeiro',
    'February': 'fevereiro',
    'March': 'março',
    'April': 'abril',
    'May': 'maio',
    'June': 'junho',
    'July': 'julho',
    'August': 'agosto',
    'September': 'setembro',
    'October': 'outubro',
    'November': 'novembro',
    'December': 'dezembro'
}
# Dicionário para traduzir os códigos de clima    
WEATHER_CODES_PT = {
        0: 'Céu limpo',
        1: 'Predominantemente limpo',
        2: 'Parcialmente nublado',
        3: 'Nublado',
        45: 'Neblina',
        48: 'Neblina com geada',
        51: 'Chuvisco: Intensidade leve',
        53: 'Chuvisco: Intensidade moderada',
        55: 'Chuvisco: Intensidade densa',
        56: 'Chuvisco congelante: Leve',
        57: 'Chuvisco congelante: Denso',
        61: 'Chuva: Intensidade leve',
        63: 'Chuva: Intensidade moderada',
        65: 'Chuva: Intensidade forte',
        66: 'Chuva congelante: Leve',
        67: 'Chuva congelante: Forte',
        71: 'Neve: Intensidade leve',
        73: 'Neve: Intensidade moderada',
        75: 'Neve: Intensidade forte',
        77: 'Grãos de neve',
        80: 'Pancadas de chuva: Leves',
        81: 'Pancadas de chuva: Moderadas',
        82: 'Pancadas de chuva: Fortes',
        85: 'Pancadas de neve: Leves',
        86: 'Pancadas de neve: Fortes',
        95: 'Tempestade: Leve ou moderada',
        96: 'Tempestade com granizo leve',
        99: 'Tempestade com granizo forte'
    }
# Coordenadas de São Paulo para a previsão do tempo
latitude = -23.5505
longitude = -46.6333

## Função para limpar os comandos, removendo verbos, etc..

In [ ]:
def extrair_termo_pesquisa(comando):
    
    # Remove o nome da assistente
    comando = re.sub(rf'{nome_assistente.lower()}\s*', '', comando)
    # Remove os verbos e espaços extras
    for verbo in verbos:
        comando = re.sub(rf'{verbo}\s*', '', comando)
    
    # Remove palavras como 'o', 'a', 'e', etc.
    comando = re.sub(r'\b(o|a|e|de|do|da|no|na)\b', '', comando).strip()
    print('limpo= '+comando)
    return comando

## Função para obter a previsão do tempo

In [ ]:

def obter_previsao_tempo():
    try:
        # Faz a requisição à API
        url = f'https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current_weather=true'
        response = requests.get(url)
        data = response.json()
        # Extrai os dados relevantes
        if 'current_weather' in data:
            temperatura = data['current_weather']['temperature']
            clima = data['current_weather']['weathercode']
            clima_str = WEATHER_CODES_PT.get(clima, 'Código de clima desconhecido')
            previsao = f'O clima está {clima_str}, a temperatura é de {temperatura}°Celsius'
            return previsao
        else:
                return frase_erro
        
    except Exception as e:
        #print(f'Erro ao obter previsão do tempo: {e}')
        return frase_erro

## Função para processar os comandos e executar as ações

In [ ]:

def processar_comando(comando):
    
    # Comando para pesquisar no Google
    if ('pesquisar' in comando or 'pesquise' in comando or
         'encontre' in comando or 'encontrar' in comando or 
         'procure' in comando or  'procurar' in comando or
         'mostre' in comando or 'mostrar' in comando):
        termo = extrair_termo_pesquisa(comando)
        print(f'Pesquisando no Google: {termo}')
        webbrowser.open(f'https://www.google.com/search?q={termo}')

    # Comando para pesquisar na Wikipedia
    elif 'wikipedia' in comando or 'wikipédia' in comando:
        termo = re.sub(r'(wikipedia|wikipédia)\s*', '', comando)  # Remove a palavra 'wikipedia'
        termo = extrair_termo_pesquisa(termo)
        print(f'Pesquisando na Wikipedia: {termo}')
        webbrowser.open(f'https://pt.wikipedia.org/wiki/{termo}')

    # Comando para abrir um vídeo no YouTube
    elif 'youtube' in comando:
        termo = re.sub(r'youtube\s*', '', comando)  # Remove a palavra 'youtube'
        termo = extrair_termo_pesquisa(termo)
        print(f'Pesquisando no YouTube: {termo}')
        webbrowser.open(f'https://www.youtube.com/results?search_query={termo}')

    # Comando para abrir a calculadora
    elif 'calculadora' in comando:
        print('Abrindo a calculadora...')
        subprocess.Popen('calc.exe')  

    # Comando para falar o dia
    elif 'dia' in comando:
        hoje = datetime.datetime.now()
        dia_semana = hoje.strftime('%A')  # Retorna o dia da semana em inglês
        dia_mes = hoje.strftime('%d')  # Retorna o dia do mês
        mes = hoje.strftime('%B')  # Retorna o mês em inglês
        # Traduz o dia da semana e o mês para português
        dia_semana_pt = DIAS_PT.get(dia_semana, dia_semana)
        mes_pt = MESES_PT.get(mes, mes)
        resposta = f'Hoje é {dia_semana_pt}, dia {dia_mes} de {mes_pt}.'
        falar(resposta)

    # Comando para falar a hora
    elif 'hora' in comando:
        agora = datetime.datetime.now()
        hora = agora.strftime('%H:%M')
        resposta = f'Agora são {hora}.'
        falar(resposta)

    # Comando para falar a previsão do tempo
    else: 
        palavras = comando.split()
        for palavra in palavras:
            if palavra in ['tempo', 'clima', 'previsão', 'chuva', 'chover', 'temperatura', 'calor', 'frio', 'sol'] :
                resposta = obter_previsao_tempo()
                falar(resposta)
                comando = ''
                break;
        if comando != '':
           falar(frase_erro_comando)


## Função para ouvir o microfone

In [ ]:

def ouvir_comando():
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.pause_threshold = 1
        r.adjust_for_ambient_noise(source, duration=1)
        audio = r.listen(source)
        try:
            texto = r.recognize_google(audio, language='pt-BR')
            return texto.lower()  # Converte para minúsculas
        except sr.UnknownValueError:
            falar(frase_nao_entendi)
            return ''
        except sr.RequestError as e:
            falar(frase_erro)
            print(f'Erro ao se comunicar com o serviço de reconhecimento de fala; {e}')
            return None



# Função principal

In [ ]:

falar(frase_ola)
while True:
    # Captura o áudio e converte em texto
    texto = ouvir_comando()
    print(f'Você disse: {texto}')
    # Verifica se o gatilho  foi detectado
    if 'sair' in texto:
        falar(frase_encerrando)
        break
    if texto and nome_assistente.lower() in texto:
        print(f'Gatilho <{nome_assistente}> detectado!')
        comando = texto.replace(nome_assistente, '').strip()  # Remove o gatilho e espaços extras
        if comando:
            print(f'Comando recebido: {comando}')
            processar_comando(comando)
        else:
            print('Nenhum comando detectado após o gatilho.')
    else:
        print(f'Gatilho <{nome_assistente}> não detectado. Aguardando...')
        #falar(frase_ajuda)

falando: Olá sou Tina sua assistente virtual
Você disse: tina
Gatilho <Tina> detectado!
Comando recebido: tina
falando: Comando não reconhecido. Tente novamente.
Você disse: que dia é hoje
Gatilho <Tina> não detectado. Aguardando...
Você disse: tina que dia é hoje
Gatilho <Tina> detectado!
Comando recebido: tina que dia é hoje
falando: Hoje é sábado, dia 22 de fevereiro.
Você disse: tina que horas
Gatilho <Tina> detectado!
Comando recebido: tina que horas
falando: Agora são 22:03.
Você disse: tina qual a previsão
Gatilho <Tina> detectado!
Comando recebido: tina qual a previsão
falando: O clima está Parcialmente nublado, a temperatura é de 22.0°Celsius
Você disse: tina abrir rock and roll no youtube
Gatilho <Tina> detectado!
Comando recebido: tina abrir rock and roll no youtube
limpo= rock and roll
Pesquisando no YouTube: rock and roll
Você disse: tina pesquisar na wikipédia frança
Gatilho <Tina> detectado!
Comando recebido: tina pesquisar na wikipédia frança
limpo= wikipédia frança
Pes